In [9]:
#Montar drive para leer archivos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
#Leer archivos csv desde el drive
import numpy as np
import pandas as pd

#Leer archivos csv desde el drive
olist_orders_dataset= pd.read_csv \
 ("/content/drive/MyDrive/Recursos DN_COM_58/olist_orders_dataset.csv", parse_dates=["order_purchase_timestamp", \
 "order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"])
olist_customers_dataset= pd.read_excel \
 ("/content/drive/MyDrive/Recursos DN_COM_58/olist_customers_dataset.xlsx")
brasil_regions= pd.read_csv \
 ("/content/drive/MyDrive/Recursos DN_COM_58/brasil_regions.csv")
olist_geolocation_dataset= pd.read_csv \
 ("/content/drive/MyDrive/Recursos DN_COM_58/olist_geolocation_dataset.csv")
olist_order_items_dataset= pd.read_csv \
 ("/content/drive/MyDrive/Recursos DN_COM_58/olist_order_items_dataset.csv", parse_dates=["shipping_limit_date"])
olist_order_payments_dataset= pd.read_csv \
 ("/content/drive/MyDrive/Recursos DN_COM_58/olist_order_payments_dataset.csv")

#Consolidar olist_geolocation_dataset y brasil_regions
df0= olist_geolocation_dataset.merge(brasil_regions, how="left", left_on="geolocation_state", right_on="abbreviation")
df0= df0[["geolocation_zip_code_prefix", "geolocation_lat", "geolocation_lng", "state_name", "region"]]
df0= df0.drop_duplicates(subset=['geolocation_zip_code_prefix'])

#Consolidar olist_customers_dataset con df0
df1= olist_customers_dataset.merge(df0, how="left", left_on="customer_zip_code_prefix", right_on="geolocation_zip_code_prefix")
df1= df1.drop(['customer_unique_id'], axis=1)

#Resumir reporte olist_order_items_dataset
df2 = pd.pivot_table(olist_order_items_dataset,index=['order_id'],aggfunc={'order_item_id':np.max, 'freight_value':np.sum,'price':np.sum})
df2.reset_index(drop= False, inplace= True)

#Consolidar olist_orders_dataset y df1
df3= pd.merge(olist_orders_dataset, df1, on=["customer_id"])

#Consolidado completo (df2 y df3)
consolidado= pd.merge(df3, df2, on=["order_id"], how="left")


In [90]:
#Importar reporte a un archivo excel
import pandas as pd
with pd.ExcelWriter("/content/drive/MyDrive/Recursos DN_COM_58/RETO/consolidado.xlsx") as writer:
    consolidado.to_excel(writer, sheet_name="consolidado", index=False)
    df0.to_excel(writer, sheet_name="df0")
    df1.to_excel(writer, sheet_name="df1")
    df2.to_excel(writer, sheet_name="df2")
    df3.to_excel(writer, sheet_name="df3")